<a href="https://colab.research.google.com/github/Rana-Altaify/Alzheimer-s_ML_Model/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import libraries**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# **Import Data**

In [ ]:
from huggingface_hub import hf_hub_download
import zipfile, os
from tensorflow import keras

repo_id = "r2n2/bestalzheimer"
filename = "alzhemier.zip"

file_path = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset")

extract_dir = "./alz_data"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(" Files extracted to:", extract_dir)
print("Folders:", os.listdir(extract_dir))

base_dir = os.path.join(extract_dir, "Combined Dataset")
train_path = os.path.join(base_dir, "train")
test_path = os.path.join(base_dir, "test")

print(" Train Path:", train_path)
print(" Test Path:", test_path)

train_dataset = keras.utils.image_dataset_from_directory(
    directory=train_path,
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    color_mode="rgb",
)

test_dataset = keras.utils.image_dataset_from_directory(
    directory=test_path,
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    color_mode="rgb",
)

print(" Datasets loaded successfully!")
print("Class Names:", train_dataset.class_names)


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

train_dir = "./alz_data/Combined Dataset/train"
test_dir = "./alz_data/Combined Dataset/test"

data = []
for folder in [train_dir, test_dir]:
    for class_name in os.listdir(folder):
        class_path = os.path.join(folder, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    data.append({
                        "filepath": os.path.join(class_path, img_name),
                        "label": class_name
                    })

df = pd.DataFrame(data)
print(" Total images:", len(df))
print(df["label"].value_counts())

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

print("\nTraining samples per class:")
print(train_df["label"].value_counts())
print("\nTesting samples per class:")
print(test_df["label"].value_counts())

train_df.to_csv("train_split.csv", index=False)
test_df.to_csv("test_split.csv", index=False)

print("\n New stratified split saved successfully!")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = preprocess_input(image)
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_df["filepath"].values, train_df["label"].astype('category').cat.codes.values)
)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_df["filepath"].values, test_df["label"].astype('category').cat.codes.values)
)

train_dataset = train_dataset.map(load_and_preprocess_image).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

class_names = list(df["label"].astype('category').cat.categories)
print("Class names:", class_names)


In [ ]:
import tensorflow as tf

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_df["filepath"].values, train_df["label"].astype('category').cat.codes.values))
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_df["filepath"].values, test_df["label"].astype('category').cat.codes.values))
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
class_counts = Counter()
for _, labels in train_dataset:
    class_counts.update(labels.numpy())
print("Class Counts:", dict(class_counts))

In [ ]:
def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

data = train_dataset.map(preprocess)
test = test_dataset.map(preprocess)

In [ ]:
transmodel = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

In [ ]:
transmodel.trainable = True
set_trainable = False
for layer in transmodel.layers:
    if layer.name == "block5a_expand_activation":
        set_trainable = True
    layer.trainable = set_trainable


In [ ]:

from tensorflow.keras import Model, layers

x = transmodel.output
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=transmodel.input, outputs=output)


In [ ]:
callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.01,
    patience=11,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
    start_from_epoch=0
)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    data,
    batch_size=32,
    epochs=59,
    validation_data=test,
    callbacks=[callback]
)


In [ ]:
ef = pd.DataFrame(history.history)
ef[['accuracy', 'val_accuracy']].plot(title="Accuracy")
ef[['loss', 'val_loss']].plot(title="Loss")

In [ ]:
model.evaluate(test)

Actual = []
predictions = []
for images, labels in test:
    Actual.extend(labels.numpy())
    pred = model.predict(images)
    predict = np.argmax(pred, axis=1)
    predictions.extend(predict)

x = classification_report(Actual, predictions, target_names=class_names)
print(x)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import os

conf_matrix = confusion_matrix(Actual, predictions)
print("Confusion Matrix:")
print(conf_matrix)

train_path = "./alz_data/Combined Dataset/train"
class_names = sorted(os.listdir(train_path))

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
correct = np.trace(conf_matrix)
incorrect = np.sum(conf_matrix) - correct

sizes = [correct, incorrect]
plt.pie(
    sizes,
    labels=["Correct", "Incorrect"],
    autopct="%1.1f%%",
    explode=[0.05, 0.05],
    startangle=45,
    colors=["#66b3ff", "#ff6666"]
)
plt.title("Prediction Accuracy")
plt.show()

print("Correct:", correct)
print("Incorrect:", incorrect)

In [ ]:
history.history['accuracy'][-1], history.history['val_accuracy'][-1]

In [ ]:
model.save("alzheimers_model.keras")


In [ ]:
import tensorflow as tf
import numpy as np
import gradio as gr
from tensorflow.keras.applications.efficientnet import preprocess_input
import nest_asyncio

model = tf.keras.models.load_model("alzheimers_model.keras")

def predict(image):
    img = tf.image.resize(image, (224, 224))
    if img.shape[-1] == 1:
        img = tf.image.grayscale_to_rgb(img)
    img = preprocess_input(img)
    img = tf.expand_dims(img, axis=0)
    preds = model.predict(img)
    class_idx = np.argmax(preds)
    confidence = float(np.max(preds))
    class_names = ["Mild Demented", "Moderate Demented", "Non Demented", "Very Mild Demented"]
    # العودة كـ dict للتوافق مع Gradio
    result = {class_names[class_idx]: confidence}
    return result

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Label(num_top_classes=1),
    title="Alzheimer MRI Classifier",
    description="Upload an MRI brain image and get the predicted class."
)

nest_asyncio.apply()
interface.launch(share=True)
